In [ ]:
import os
import h5py
import pickle
import numpy as np
from utils import LatLonStandardScaler, StandardScaler

from sgptools.utils.gpflow import *
from sgptools.utils.metrics import *
from sgptools.utils.misc import polygon2candidates

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def plot_results(X_candidates, y_candidates, var, 
                 X, waypoints,
                 title=None, fname=None, 
                 figsize=(10, 4), update_waypoint=None):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=figsize)

    # Reconstruction
    map = ax[0].scatter(X_candidates[:, 1], X_candidates[:, 0],
                        c=y_candidates)
    ax[0].scatter(X[:, 1], X[:, 0], c='r', s=0.5, marker='.', label='Data')
    ax[0].plot(waypoints[:, 1], waypoints[:, 0], marker='o', c='r', label='Planned Path')
    if update_waypoint is not None:
        ax[0].scatter(waypoints[update_waypoint, 1], waypoints[update_waypoint, 0],
                      marker='o', c='g', label='Update Waypoint', zorder=10)
    ax[0].set_xlabel("X")
    ax[0].set_ylabel("Y")
    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(map, cax=cax, orientation='vertical')
    ax[0].set_title("Predicted Bathymetry")
    ax[0].set_aspect('equal')
    ax[0].legend()


    # Prediction Variance
    map = ax[1].scatter(X_candidates[:, 1], X_candidates[:, 0],
                        c=var)
    ax[1].set_xlabel("X")
    ax[1].set_yticks([])
    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(map, cax=cax, orientation='vertical')
    ax[1].set_title("Prediction Variance")
    ax[1].set_aspect('equal')

    if title is not None:
        plt.suptitle(title)
    if fname is not None:
        plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
mission_log = "../launch/data/IPP-mission-2025-07-08-21-43-03"
num_samples = 50000

# data file
fname = os.path.join(mission_log,
                     "mission-log.hdf5")

# load data
with h5py.File(fname, "r") as f:
    fence_vertices = f["fence_vertices"][:].astype(float)
    X = f["X"][:].astype(float)
    y = f["y"][:].astype(float)

    waypoints = {}
    for key in f.keys():
        if "waypoints" in key:
            waypoints[key] = f[key][:].astype(float)

print(f'Mission Log: {mission_log}')
print(f'Number of data samples: {X.shape[0]}')
print(f'Number of reconstruction samples: {num_samples}')

# Normalize the candidates
X_candidates = polygon2candidates(fence_vertices, 
                                  num_samples=num_samples)
X_scaler = LatLonStandardScaler()
X_scaler.fit(X_candidates)
X_scaler.scale_ *= 0.35
X_candidates = X_scaler.transform(X_candidates)
y_scaler = StandardScaler()
y_scaler.fit(y)

y = y_scaler.transform(y)
X = X_scaler.transform(X)

# Train GP only if pretrained weights are unavailable
fname = os.path.join(mission_log, f"RBFParams.pkl")
if os.path.exists(fname):
    with open(fname, 'rb') as handle:
        params = pickle.load(handle)
    max_steps = 0
    print('Found pre-trained parameters')
else:
    max_steps = 1000
    params = None
    print('Training from scratch')

kernel = gpflow.kernels.RBF(lengthscales=0.1, 
                            variance=0.5)
_, _, _, gpr_gt = get_model_params(X, y,
                                   kernel=kernel,
                                   return_model=True,
                                   train_inducing_pts=True,
                                   max_steps=max_steps,
                                   verbose=True)
# Load pre-trained parameters
if params is not None:
    gpflow.utilities.multiple_assign(gpr_gt.kernel, params['kernel'])
    gpflow.utilities.multiple_assign(gpr_gt.likelihood, params['likelihood'])

candidates_y, var = gpr_gt.predict_f(X_candidates)
candidates_y = candidates_y.numpy()
var = var.numpy()

for key in waypoints.keys():
    update_waypoint = int(key.split('_')[1][:-9])-1
    update_waypoint = np.max([update_waypoint, 0])
    print(f"Update Waypoint: {update_waypoint}")
    plot_results(X_scaler.inverse_transform(X_candidates), 
                 y_scaler.inverse_transform(candidates_y), 
                 var, X_scaler.inverse_transform(X), waypoints[key], "",
                 os.path.join(mission_log, f"UpdateWaypoint{key.split('_')[1][:-9]}.png"),
                 update_waypoint=update_waypoint,
                 figsize=(18, 4))